In [1]:
import io
import xml.etree.ElementTree as ET
import pandas as pd
import helper
from sklearn.feature_extraction.text import CountVectorizer
import gensim
import numpy as np
import os
#from gensim import corpora, models, similarities 

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
%install_ext https://raw.github.com/cpcloud/ipython-autotime/master/autotime.py
%load_ext autotime

Installed autotime.py. To use it, type:
  %load_ext autotime


/home/jshoun01/.virtualenvs/py27/local/lib/python2.7/site-packages/IPython/core/magics/extension.py:47: UserWarning: %install_ext` is deprecated, please distribute your extension(s)as a python packages.
  "as a python packages.", UserWarning)


In [4]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

time: 1.04 ms


In [5]:
ls data

data@
time: 832 ms


In [6]:
data_dir='data'

time: 145 ms


## Tags

In [7]:
tags_df = helper.get_xml_file_as_df(os.path.join(data_dir,'Tags.xml'))

time: 1.06 s


In [8]:
print (len(tags_df))
tags_df.head()


5056


,Count,ExcerptPostId,Id,TagName,WikiPostId
0,206,392824,203,firmware,392823
1,993,320795,204,upgrade,320794
2,13643,244736,205,networking,244735
3,3468,257013,206,audio,257012
4,2825,256215,207,video,256214


time: 384 ms


In [9]:
#posts_df = helper.get_xml_file_as_df(os.path.join(data_dir,'Posts.xml'),max_num_children=1000)
posts_df = helper.get_xml_file_as_df(os.path.join(data_dir,'Posts.xml'))

time: 1min 45s


In [10]:
posts_df.head()

,AcceptedAnswerId,AnswerCount,Body,ClosedDate,CommentCount,CommunityOwnedDate,CreationDate,FavoriteCount,Id,LastActivityDate,...,LastEditorDisplayName,LastEditorUserId,OwnerDisplayName,OwnerUserId,ParentId,PostTypeId,Score,Tags,Title,ViewCount
0,727273,10,<p>A Vista virtual machine I use only has a 10...,NaN,9,NaN,2009-07-15T06:27:46.723,69,1,2014-10-18T07:23:25.843,...,NaN,73637,NaN,1,NaN,1,132,<windows><disk-space><winsxs>,"Why does the /winsxs folder grow so large, and...",70584
1,NaN,6,<p>I used to record important technical meetin...,NaN,0,NaN,2009-07-08T16:32:47.817,3,24,2012-08-08T21:06:48.087,...,NaN,36744,essamSALAH,NaN,NaN,1,5,<audio><audio-recording><camtasia>,How can I remove noise from an audio recording?,3654
2,NaN,NaN,<p>You could use a filter on the recording. If...,NaN,0,NaN,2009-07-08T16:34:52.867,NaN,25,2009-07-15T06:48:44.300,...,NaN,NaN,Burkhard,9637,24,2,0,NaN,NaN,NaN
3,NaN,NaN,<p>try out something like:</p>\n\n<p>audible (...,NaN,0,NaN,2009-07-08T16:59:00.587,NaN,26,2009-07-15T06:48:44.300,...,NaN,NaN,Jas Panesar,23242,24,2,0,NaN,NaN,NaN
4,NaN,NaN,<p>You definitely need some sort of software t...,NaN,1,NaN,2009-07-08T17:15:59.477,NaN,27,2009-07-15T06:48:44.300,...,NaN,NaN,T Pops,45675,24,2,0,NaN,NaN,NaN


time: 84 ms


In [11]:
posts_df.columns

Index([u'AcceptedAnswerId', u'AnswerCount', u'Body', u'ClosedDate',
       u'CommentCount', u'CommunityOwnedDate', u'CreationDate',
       u'FavoriteCount', u'Id', u'LastActivityDate', u'LastEditDate',
       u'LastEditorDisplayName', u'LastEditorUserId', u'OwnerDisplayName',
       u'OwnerUserId', u'ParentId', u'PostTypeId', u'Score', u'Tags', u'Title',
       u'ViewCount'],
      dtype='object')

time: 388 ms


## Process Text

In [12]:
# strip html tags
posts_df['BodyCleaned']=posts_df['Body'].apply(lambda x : helper.strip_tags(x))
# tokenize text
#posts_df['BodyCleaned']=posts_df['BodyCleaned'].apply(lambda x : helper.tokenize_and_stem(x))
posts_df['BodyCleaned']=posts_df['BodyCleaned'].apply(lambda x : helper.tokenize_and_stem(x,stemmer=None))

# remove stop words
posts_df['BodyCleaned']=posts_df['BodyCleaned'].apply(lambda x : helper.remove_stopwords(x))



time: 29min 1s


In [13]:
posts_df.head()

,AcceptedAnswerId,AnswerCount,Body,ClosedDate,CommentCount,CommunityOwnedDate,CreationDate,FavoriteCount,Id,LastActivityDate,...,LastEditorUserId,OwnerDisplayName,OwnerUserId,ParentId,PostTypeId,Score,Tags,Title,ViewCount,BodyCleaned
0,727273,10,<p>A Vista virtual machine I use only has a 10...,NaN,9,NaN,2009-07-15T06:27:46.723,69,1,2014-10-18T07:23:25.843,...,73637,NaN,1,NaN,1,132,<windows><disk-space><winsxs>,"Why does the /winsxs folder grow so large, and...",70584,"[A, Vista, virtual, machine, I, use, GB, virtu..."
1,NaN,6,<p>I used to record important technical meetin...,NaN,0,NaN,2009-07-08T16:32:47.817,3,24,2012-08-08T21:06:48.087,...,36744,essamSALAH,NaN,NaN,1,5,<audio><audio-recording><camtasia>,How can I remove noise from an audio recording?,3654,"[I, used, record, important, technical, meetin..."
2,NaN,NaN,<p>You could use a filter on the recording. If...,NaN,0,NaN,2009-07-08T16:34:52.867,NaN,25,2009-07-15T06:48:44.300,...,NaN,Burkhard,9637,24,2,0,NaN,NaN,NaN,"[You, could, use, filter, recording, If, I, re..."
3,NaN,NaN,<p>try out something like:</p>\n\n<p>audible (...,NaN,0,NaN,2009-07-08T16:59:00.587,NaN,26,2009-07-15T06:48:44.300,...,NaN,Jas Panesar,23242,24,2,0,NaN,NaN,NaN,"[try, something, like, audible, mac, soundforg..."
4,NaN,NaN,<p>You definitely need some sort of software t...,NaN,1,NaN,2009-07-08T17:15:59.477,NaN,27,2009-07-15T06:48:44.300,...,NaN,T Pops,45675,24,2,0,NaN,NaN,NaN,"[You, definitely, need, sort, software, filter..."


time: 106 ms


In [14]:
len(posts_df)

666887

time: 349 ms


In [15]:
texts = posts_df['BodyCleaned']

time: 435 ms


In [16]:
#create a Gensim dictionary from the texts
dictionary = gensim.corpora.Dictionary(texts)

#remove extremes (similar to the min/max df step used when creating the tf-idf matrix)
dictionary.filter_extremes(no_below=10, no_above=0.8)

#convert the dictionary to a bag of words corpus for reference
corpus = [dictionary.doc2bow(text) for text in texts]

time: 1min 40s


In [17]:
corpus[0][:5]

[(343, 2), (633, 1), (1136, 1), (1340, 2), (1356, 1)]

time: 2.42 ms


In [23]:
#gensim.models.LdaMulticore
"""
lda = gensim.models.LdaModel(corpus, num_topics=5, 
                            id2word=dictionary, 
                            update_every=5, 
                            chunksize=10000,
                            passes=100)
"""

num_topics = 50

lda = gensim.models.LdaMulticore(corpus, num_topics=50, 
                            id2word=dictionary)

time: 6min 11s


In [27]:
lda.show_topics(50)

[u"0.114*I + 0.026*key + 0.020*n't + 0.010*keys + 0.009*use + 0.009*'m + 0.009*Windows + 0.009*keyboard + 0.008*like + 0.007*way",
 u"0.054*folder + 0.031*files + 0.028*file + 0.021*I + 0.016*folders + 0.012*delete + 0.010*n't + 0.010*C + 0.008*If + 0.008*Windows",
 u"0.027*I + 0.010*n't + 0.009*'s + 0.008*use + 0.008*gpg + 0.006*would + 0.006*file + 0.006*You + 0.006*LVM + 0.006*using",
 u"0.028*I + 0.024*Chrome + 0.020*page + 0.018*browser + 0.018*Google + 0.014*Firefox + 0.010*use + 0.010*web + 0.010*'s + 0.008*proxy",
 u'0.185*gt + 0.134*lt + 0.011*set + 0.008*string + 0.008*file + 0.006*name + 0.006*I + 0.005*character + 0.005*C + 0.005*The',
 u'0.024*C + 0.020*debug1 + 0.019*directory + 0.018*I + 0.017*files + 0.017*key + 0.015*file + 0.010*git + 0.009*ssh + 0.007*user',
 u"0.036*I + 0.024*task + 0.023*n't + 0.021*registry + 0.015*Windows + 0.010*time + 0.007*shutdown + 0.006*like + 0.006*settings + 0.006*use",
 u"0.083*I + 0.024*n't + 0.015*problem + 0.012*time + 0.008*windows +

time: 374 ms


In [25]:
topics_matrix = lda.show_topics(formatted=False, num_words=20)
topics_matrix = np.array(topics_matrix)

topic_words = topics_matrix[:,:,1]
for i in topic_words:
    print([str(word) for word in i])
    print()

['I', "'s", "n't", 'like', 'know', 'use', 'system', "'m", 'It', 'payload', 'would', 'malware', 'need', 'using', 'Windows', 'If', "'d", 'really', 'want', 'software']
()
['I', 'document', 'IPv6', 'use', 'Mar', 'Word', 'Nov', 'N/A', 'Reply', "n't", 'convert', 'The', 'bytes=32', 'VLC', "'s", 'using', 'NetworkManager', 'du', 'like', 'IPv4']
()
['I', 'would', 'cable', "n't", 'one', 'use', "'s", 'power', 'like', 'need', 'The', 'It', 'get', 'If', 'could', 'case', 'computer', 'You', 'signal', 'may']
()
['keyboard', 'I', "'s", 'layout', "n't", 'The', 'one', 'like', 'would', 'If', 'USB', 'use', 'keyboards', 'Red', 'keys', 'Vim', 'It', 'work', 'This', 'using']
()
['I', "n't", 'problem', 'laptop', 'tried', 'computer', 'Windows', "'m", 'fine', "'ve", 'The', 'My', 'using', 'get', 'When', 'PC', 'card', "'s", 'working', 'work']
()
["'s", "n't", 'If', 'could', 'data', 'would', 'may', 'security', 'might', "'re", 'The', 'You', 'see', 'This', 'encrypted', 'using', 'try', 'encryption', 'It', 'computer']
()


In [26]:
lda.show_topics()

[u'0.045*server + 0.039*DNS + 0.018*domain + 0.016*servers + 0.013*name + 0.010*I + 0.009*use + 0.008*hosts + 0.007*IP + 0.007*hostname',
 u'0.047*drive + 0.039*I + 0.034*disk + 0.030*partition + 0.020*hard + 0.017*data + 0.012*space + 0.011*drives + 0.010*SSD + 0.009*partitions',
 u"0.135*I + 0.015*email + 0.013*like + 0.012*would + 0.011*'m + 0.011*Outlook + 0.010*know + 0.009*way + 0.009*message + 0.009*n't",
 u"0.137*amp + 0.009*I + 0.009*n't + 0.007*You + 0.006*using + 0.006*terminal + 0.006*'s + 0.006*If + 0.005*like + 0.005*printf",
 u"0.029*I + 0.027*driver + 0.026*device + 0.017*drivers + 0.014*battery + 0.012*USB + 0.012*laptop + 0.012*power + 0.009*card + 0.008*'s",
 u"0.036*I + 0.024*task + 0.023*n't + 0.021*registry + 0.015*Windows + 0.010*time + 0.007*shutdown + 0.006*like + 0.006*settings + 0.006*use",
 u'0.015*Visual + 0.015*TCP + 0.014*Java + 0.014*java + 0.012*Studio + 0.012*event + 0.012*May + 0.010*time + 0.009*Event + 0.008*If',
 u'0.024*C + 0.020*debug1 + 0.019*di

time: 414 ms
